In [1]:
! pip install openai langchain
! pip install python_dotenv
! pip install --quiet -U langchain_openai langchain_core langchain_community
! pip install pypdf
! pip install faiss-cpu
! pip install scholarly
! pip install gradio

In [16]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

In [17]:
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

def chat_with_model(user_input):
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=user_input)
    ]
    
    response = openai_model(messages)
    
    return response.content

interface = gr.Interface(
    fn=chat_with_model,
    inputs=gr.Textbox(lines=2, label="Enter your question"),
    outputs="text",
    title="Chat with AI Assistant",
    description="Ask questions and receive helpful responses from the AI assistant."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [4]:
import gradio as gr
import requests
from scholarly import scholarly
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

def download_pdf(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        with open(filename, 'wb') as f:
            f.write(response.content)
        return f"Downloaded PDF: {filename}"
    except Exception as e:
        return f"Failed to download PDF: {e}"

global_vector_store = None

def store_pdf_in_vector_space(filename):
    global global_vector_store  
    loader = PyPDFLoader(filename)
    documents = loader.load()

    embeddings = OpenAIEmbeddings()

    global_vector_store = FAISS.from_documents(documents, embeddings)

    global_vector_store.save_local("vector_store")
    return "PDF successfully stored in vector space."

def research_assistant(query, download_and_store):
    try:
        search_query = scholarly.search_pubs(query)
        paper = next(search_query)

        title = paper.get('bib', {}).get('title', 'Title not available')
        abstract = paper.get('bib', {}).get('abstract', 'Abstract not available')
        authors = paper.get('bib', {}).get('author', 'Authors not available')
        pub_year = paper.get('bib', {}).get('pub_year', 'Publication year not available')
        pub_url = paper.get('pub_url', 'URL not available')

        if isinstance(authors, list):
            authors = ', '.join(authors)

        result = f"Title: {title}\n\nAbstract: {abstract}\n\nAuthors: {authors}\n\nPublication Year: {pub_year}\n\nFull Paper URL: {pub_url}"

        if download_and_store and pub_url.endswith(".pdf"): 
            pdf_filename = "downloaded_paper.pdf"
            download_status = download_pdf(pub_url, pdf_filename)

            if "Downloaded PDF" in download_status:
                store_status = store_pdf_in_vector_space(pdf_filename)
                result += f"\n\n{download_status}\n{store_status}"
            else:
                result += f"\n\n{download_status}"
        elif download_and_store:
            result += "\n\nThe paper is not directly available as a PDF."

    except StopIteration:
        result = "No results found for the query."

    return result

interface = gr.Interface(
    fn=research_assistant,  
    inputs=[
        gr.Textbox(lines=2, label="Enter Your Query"), 
        gr.Checkbox(label="Download and store full text (PDF)")  
    ],
    outputs="text",          
    title="AI Research Assistant with PDF Storage",
    description="Enter your search query to find relevant research papers and summaries. Optionally download the full text (PDF) and store it in a vector space."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\HP\AppData\Local\Temp\ipykernel_9320\160970845.py:26: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [18]:
import gradio as gr
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=openai_model,           
    chain_type="stuff",         
    retriever=global_vector_store.as_retriever()  
)

def query_retriever(question):
    response = qa_chain.run(question)
    return response

interface = gr.Interface(
    fn=query_retriever,         
    inputs=gr.Textbox(lines=2, label="Enter Your Question"), 
    outputs="text",             
    title="AI Research Assistant",  
    description="Ask questions about your research papers and get summarized answers from the AI."
)

interface.launch()

# "What are the key findings of the research paper?"

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [7]:
import gradio as gr
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

vector_store = FAISS.load_local("vector_store", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever()

summarization_prompt = """
Summarize the following document section in a concise and clear manner:

{document}

Summary:
"""
prompt = PromptTemplate(input_variables=["document"], template=summarization_prompt)

summarization_chain = LLMChain(llm=openai_model, prompt=prompt)

def summarize_query(query):
    retrieved_docs = retriever.get_relevant_documents(query)
    
    if not retrieved_docs:
        return "No relevant documents found."

    document_content = retrieved_docs[0].page_content
    summary = summarization_chain.run(document=document_content)

    return summary

interface = gr.Interface(
    fn=summarize_query,  
    inputs=gr.Textbox(lines=2, label="Enter your query"),  
    outputs="text",  
    title="Document Summarization Assistant",
    description="Enter a query to summarize the relevant sections of your documents."
)

interface.launch()

C:\Users\HP\AppData\Local\Temp\ipykernel_9320\1946687300.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarization_chain = LLMChain(llm=openai_model, prompt=prompt)


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


C:\Users\HP\AppData\Local\Temp\ipykernel_9320\1946687300.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [15]:
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

vector_store = FAISS.load_local("vector_store", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()


# memory to store conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# prompt for generating follow-up questions
question_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="Given the conversation so far: {chat_history}, and the question: {question}, generate a follow-up question if necessary."
)

question_generator = LLMChain(
    llm=openai_model,
    prompt=question_prompt
)

qa_chain = load_qa_chain(llm=openai_model, chain_type="stuff")

# conversational retrieval chain with the required components
conversation_chain = ConversationalRetrievalChain(
    retriever=retriever,
    memory=memory,
    combine_docs_chain=qa_chain,
    question_generator=question_generator,
    verbose=True 
)

def chat_with_agent(user_input):
    retrieved_docs = retriever.get_relevant_documents(user_input)
    print("Retrieved Documents:", [doc.page_content for doc in retrieved_docs])  # Print retrieved document content
    
    response = conversation_chain.invoke({"question": user_input})
    return response['answer']


interface = gr.Interface(
    fn=chat_with_agent,
    inputs=gr.Textbox(lines=2, label="Enter your query"),
    outputs="text",
    title="Conversational Agent with Document Focus",
    description="Chat with an AI assistant that retrieves information from documents while maintaining conversation context."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Retrieved Documents: ['Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base\nmodel. All metrics are on the English-to-German translation development set, newstest2013. Listed\nperplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to\nper-word perplexities.\nN d modeldffh d kdvPdropϵlstrain PPL BLEU params\nsteps (dev) (dev)×106\nbase 6 512 2048 8 64 64 0.1 0.1 100K 4.92 25.8 65\n(A)1 512 512 5.29 24.9\n4 128 128 5.00 25.5\n16 32 32 4.91 25.8\n32 16 16 5.01 25.4\n(B)16 5.16 25.1 58\n32 5.01 25.4 60\n(C)2 6.11 23.7 36\n4 5.19 25.3 50\n8 4.88 25.5 80\n256 32 32 5.75 24.5 28\n1024 128 128 4.66 26.0 168\n1024 5.12 25.4 53\n4096 4.75 26.2 90\n(D)0.0 5.77 24.6\n0.2 4.95 25.5\n0.0 4.67 25.3\n0.2 5.47 25.7\n(E) positional embedding instead of sinusoids 4.92 25.7\nbig 6 1024 4096 16 0.3 300K 4.33 26.4 213\nIn Table 3 rows (B), we observe that reducing the attention key size dkhurts model quality. This\ns


> Finished chain.
Retrieved Documents: ['Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base\nmodel. All metrics are on the English-to-German translation development set, newstest2013. Listed\nperplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to\nper-word perplexities.\nN d modeldffh d kdvPdropϵlstrain PPL BLEU params\nsteps (dev) (dev)×106\nbase 6 512 2048 8 64 64 0.1 0.1 100K 4.92 25.8 65\n(A)1 512 512 5.29 24.9\n4 128 128 5.00 25.5\n16 32 32 4.91 25.8\n32 16 16 5.01 25.4\n(B)16 5.16 25.1 58\n32 5.01 25.4 60\n(C)2 6.11 23.7 36\n4 5.19 25.3 50\n8 4.88 25.5 80\n256 32 32 5.75 24.5 28\n1024 128 128 4.66 26.0 168\n1024 5.12 25.4 53\n4096 4.75 26.2 90\n(D)0.0 5.77 24.6\n0.2 4.95 25.5\n0.0 4.67 25.3\n0.2 5.47 25.7\n(E) positional embedding instead of sinusoids 4.92 25.7\nbig 6 1024 4096 16 0.3 300K 4.33 26.4 213\nIn Table 3 rows (B), we observe that reducing the attention key size dkhurts mod

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')



> Finished chain.
